In [28]:
import requests
import time
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
from datetime import date
import numpy as np


In [13]:
def convert_blocktime(blocktime, delta = 6):
    if blocktime is None:
        return None
    else:
        timestr = blocktime.replace('T',' ').replace('Z','')
        return datetime.strptime(timestr[:-delta], "%Y-%m-%d %H:%M:%S")

In [14]:
def get_mixnodes(used_api):
    
    if used_api=='nymtech_api':
        request_str = 'https://validator.nymtech.net/api/v1/mixnodes/'
    elif used_api=='api_guru':
        request_str = 'https://mixnet.api.explorers.guru/api/mixnodes'
    
    print(request_str)
    return requests.get(request_str).json()

In [15]:
def get_mixnode_balance(mix_id):
    request_str='https://mixnet.api.explorers.guru/api/accounts/{mix_id}/balance'.format(mix_id=mix_id)
    print(request_str)
    return requests.get(request_str).json()

def get_account_pledge(account_id):
    request_str='https://mixnet.api.explorers.guru/api/accounts/{account_id}/bondings'.format(account_id=account_id)
    print(request_str)
    return requests.get(request_str).json()

In [16]:
def get_network_rewards_history(min_epoch,max_epoch):
    dct_out={'epoch':[],
            'rewards':[],
            'apy':[]}
    for i in range(min_epoch,max_epoch):
        request_str = 'https://mixnet.api.explorers.guru/api/mixnodes/apy/{epoch}'.format(epoch=str(i))
        print(request_str)
        dct_rewards = requests.get(request_str).json()
        if dct_rewards:
            dct_out['epoch'].append(i)
            if 'rewards1h' in dct_rewards.keys():
                dct_out['rewards'].append(dct_rewards['rewards1h'])
            else:
                dct_out['rewards'].append(0)
            if 'apy' in dct_rewards.keys():
                dct_out['apy'].append(dct_rewards['apy'])
            else:
                dct_out['apy'].append(0)
        else:
            print('no dct for '+str(i))
    return pd.DataFrame.from_dict(dct_out)

In [17]:
def get_mixnode_reward_estimate(mix_id):
    request_str = 'https://mixnet.api.explorers.guru/api/mixnodes/{mix_id}/estimated_reward'.format(mix_id=mix_id)
    print(request_str)
    return requests.get(request_str).json()
    

In [18]:
def get_mixnode_stats(mix_ip_address):
    dct_out={'packets_received_since_startup':None,
    'packets_sent_since_startup':None,
    'packets_explicitly_dropped_since_startup':None}
        
    request_str = 'https://{mix_ip_address}:8000/stats'.format(mix_ip_address=mix_ip_address)
    print(request_str)

    try:
        result = requests.get(request_str)
    except requests.exceptions.RequestException as e:  
        print(e)
        result = None
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
        result = None

    print(result)
    if result:
        return result.json()
    else:
        return dct_out
    
    

In [19]:
def get_uptime(mix_id):
    request_str = 'https://mixnet.api.explorers.guru/api/mixnodes/{mix_id}/uptime'.format(mix_id=mix_id)
    print(request_str)
    return requests.get(request_str).json()
    
def get_block_date(blockheight,**kwargs):
    blockcount = kwargs.get('blockcount',1)
    request_str = 'https://mixnet.api.explorers.guru/api/blocks?count={blockcount}&height={blockheight}'.format(blockcount=blockcount,blockheight=blockheight)
    print(request_str)
    return requests.get(request_str).json()

In [20]:
def get_price_history():
    request_str='https://mixnet.api.explorers.guru/api/market/history'
    print(request_str)
    dct_prices = requests.get(request_str).json()
    if dct_prices:
        df_prices = pd.DataFrame.from_dict(dct_prices)
        df_prices['datestr']=[convert_blocktime(df_prices.iloc[i].timestamp, delta=4) for i in range(0,len(df_prices))]
        return df_prices.sort_values(by='datestr',ascending=False)
    else:
        print('error pulling price history')
        return pd.DataFrame.from_dict({})

In [23]:
def get_price_data():
    dct_out={}
    request_str='https://mixnet.api.explorers.guru/api/market/price'
    print(request_str)
    dct_prices = requests.get(request_str).json()
    if dct_prices:
        cnt = 0
        for k in dct_prices.keys():
            if cnt==0:
                dct_out[k]=[]
            
            dct_out[k].append(dct_prices[k])
        
        df_prices = pd.DataFrame.from_dict(dct_out)
        df_prices['datestr']=[convert_blocktime(df_prices.iloc[i].timestamp, delta=4) for i in range(0,len(df_prices))]
        return df_prices
    else:
        print('error pulling price history')
        return pd.DataFrame.from_dict({})
    

def read_mixnodes(**kwargs):
    
    conversion_factor = kwargs.get('conversion_factor',np.power(10,6))
    start_node = kwargs.get('start_node',0)
    node_data = get_mixnodes('nymtech_api')
    df_node_data_2 = pd.DataFrame.from_dict(get_mixnodes('api_guru'))
    


    dct_out=  {} 
    for i in range(start_node,len(node_data)):
        output_balance = get_mixnode_balance(node_data[i]['bond_information']['owner'])
        
        mix_id=node_data[i]['bond_information']['mix_id']
        mix_ip_address=node_data[i]['bond_information']['mix_node']['host']
        
        estimates = get_mixnode_reward_estimate(mix_id)
        uptime= get_uptime(mix_id)
        stats = get_mixnode_stats(mix_ip_address)

        if i==0:
            dct_out['id']=[]
            dct_out['owner']=[]  
            dct_out['rank']=[] 
            dct_out['status']=[] 
            dct_out['verify']=[] 
            dct_out['warning']=[] 
            dct_out['warning_description']=[] 
            dct_out['saturation']=[]
            dct_out['current_bond']=[]
            dct_out['last_touched']=[]
            dct_out['stake_share']=[]
            dct_out['original_pledge']=[]
            dct_out['layer']=[]
            dct_out['mix_node_ip']=[]
            dct_out['mix_node_sphinx_key']=[]
            dct_out['mix_node_identity_key']=[]
            dct_out['mix_node_version']=[]
            dct_out['proxy']=[]
            dct_out['bonding_height']=[]
            dct_out['is_unbonding']=[]
            dct_out['profit_margin']=[]
            dct_out['cost_hour']=[]
            dct_out['delegate_stake']=[]
            dct_out['reward_total']=[]
            dct_out['last_rewarded_epoch']=[]
            dct_out['unique_delegations']=[]
            dct_out['apy']=[]
            dct_out['total_node_reward_estimate']=[]
            dct_out['operator_estimate']=[]
            dct_out['delegates_estimate']=[]
            dct_out['uptime_most_recent']=[]
            dct_out['uptime_last_hour']=[]
            dct_out['uptime_last_day']=[]
            dct_out['packets_received_since_startup']=[]
            dct_out['packets_sent_since_startup']=[]
            dct_out['packets_explicitly_dropped_since_startup']=[]

            for k in output_balance.keys():
                dct_out[k]=[]

        dct_out['id'].append(mix_id)
        dct_out['owner'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['address'].iloc[0])
        dct_out['rank'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['rank'].iloc[0]) 
        dct_out['status'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['status'].iloc[0])
        dct_out['saturation'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['saturationPercent'].iloc[0])    
        dct_out['current_bond'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['bond'].iloc[0]/conversion_factor)    
        dct_out['original_pledge'].append(float(node_data[i]['bond_information']['original_pledge']['amount'])/conversion_factor)
        dct_out['delegate_stake'].append(float(node_data[i]['rewarding_details']['delegates'])/conversion_factor)
        dct_out['last_touched'].append(convert_blocktime(df_node_data_2[df_node_data_2['mixId']==mix_id]['touched'].iloc[0],delta=4))    
        dct_out['stake_share'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['totalStakePercent'].iloc[0])
        dct_out['apy'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['apy'].iloc[0])
        dct_out['total_node_reward_estimate'].append(float(estimates['total_node_reward'])/conversion_factor)
        dct_out['operator_estimate'].append(float(estimates['operator'])/conversion_factor)
        dct_out['delegates_estimate'].append(float(estimates['delegates'])/conversion_factor)
        dct_out['profit_margin'].append(float(node_data[i]['rewarding_details']['cost_params']['profit_margin_percent']))
        dct_out['cost_hour'].append(float(node_data[i]['rewarding_details']['cost_params']['interval_operating_cost']['amount'])/conversion_factor/24/31) #check if they cahnge the month-days
        dct_out['reward_total'].append(float(node_data[i]['rewarding_details']['total_unit_reward'])/conversion_factor)
        dct_out['last_rewarded_epoch'].append(int(node_data[i]['rewarding_details']['last_rewarded_epoch']))
        dct_out['unique_delegations'].append(int(node_data[i]['rewarding_details']['unique_delegations']))
        dct_out['uptime_most_recent'].append(uptime['most_recent'])
        dct_out['uptime_last_hour'].append(uptime['last_hour'])
        dct_out['uptime_last_day'].append(uptime['last_day'])
        dct_out['packets_received_since_startup'].append(stats['packets_received_since_startup'])
        dct_out['packets_sent_since_startup'].append(stats['packets_sent_since_startup'])
        dct_out['packets_explicitly_dropped_since_startup'].append(stats['packets_explicitly_dropped_since_startup'])
        dct_out['layer'].append(int(node_data[i]['bond_information']['layer']))
        dct_out['mix_node_ip'].append(mix_ip_address)
        dct_out['mix_node_sphinx_key'].append(node_data[i]['bond_information']['mix_node']['sphinx_key'])
        dct_out['mix_node_identity_key'].append(node_data[i]['bond_information']['mix_node']['identity_key'])
        dct_out['mix_node_version'].append(node_data[i]['bond_information']['mix_node']['version'])
        dct_out['proxy'].append(node_data[i]['bond_information']['proxy'])
        dct_out['bonding_height'].append(int(node_data[i]['bond_information']['bonding_height']))
        dct_out['is_unbonding'].append(int(node_data[i]['bond_information']['is_unbonding']))
        dct_out['verify'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['verify'].iloc[0])
        dct_out['warning'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['warning'].iloc[0])
        dct_out['warning_description'].append(df_node_data_2[df_node_data_2['mixId']==mix_id]['warningDescription'].iloc[0])    
        
        for k in output_balance.keys():
            dct_out[k].append(float(output_balance[k]['amount'])/conversion_factor)
        

    return pd.DataFrame.from_dict(dct_out)


In [11]:
#network rewards per epoch, mapping to dates via relationship 1 epoch ~ 1h
df_reward_history = get_network_rewards_history(1,4000)

https://mixnet.api.explorers.guru/api/mixnodes/apy/3860
https://mixnet.api.explorers.guru/api/mixnodes/apy/3861
https://mixnet.api.explorers.guru/api/mixnodes/apy/3862
https://mixnet.api.explorers.guru/api/mixnodes/apy/3863
https://mixnet.api.explorers.guru/api/mixnodes/apy/3864
https://mixnet.api.explorers.guru/api/mixnodes/apy/3865
https://mixnet.api.explorers.guru/api/mixnodes/apy/3866
https://mixnet.api.explorers.guru/api/mixnodes/apy/3867
https://mixnet.api.explorers.guru/api/mixnodes/apy/3868
https://mixnet.api.explorers.guru/api/mixnodes/apy/3869
https://mixnet.api.explorers.guru/api/mixnodes/apy/3870
https://mixnet.api.explorers.guru/api/mixnodes/apy/3871
https://mixnet.api.explorers.guru/api/mixnodes/apy/3872
https://mixnet.api.explorers.guru/api/mixnodes/apy/3873
https://mixnet.api.explorers.guru/api/mixnodes/apy/3874
https://mixnet.api.explorers.guru/api/mixnodes/apy/3875
https://mixnet.api.explorers.guru/api/mixnodes/apy/3876
https://mixnet.api.explorers.guru/api/mixnodes/a

In [ ]:
#price data from coingecko
df_prices = pd.read_csv('nym-usd-max.csv')
df_prices['datestr']=[convert_blocktime(df_prices.snapped_at.iloc[i], delta=4) for i in range(0, len(df_prices))]
df_prices['dateday']=[date(df_prices.datestr.iloc[i].year, df_prices.datestr.iloc[i].month, df_prices.datestr.iloc[i].day)for i in range(0, len(df_prices))]
df_reward_history['datestr']=[datetime.strptime('2023-01-13 12:00:00','%Y-%m-%d %H:%M:%S')-(691-df_reward_history.epoch.iloc[i])*timedelta(hours=1) for i in range(0, len(df_reward_history))]
df_reward_history['dateday']=[date(df_reward_history.datestr.iloc[i].year, df_reward_history.datestr.iloc[i].month, df_reward_history.datestr.iloc[i].day)for i in range(0, len(df_reward_history))]
df_reward_history['year_month']=[str(df_reward_history.datestr.iloc[i].year) + "-" + str(df_reward_history.datestr.iloc[i].month) for i in range(0, len(df_reward_history))]
df_reward_history_month =df_reward_history.groupby('year_month').agg(
 num_epochs = ('datestr','count'),
    sum_rewards=('rewards','sum')   
).reset_index()
df_reward_history_days = df_reward_history.groupby('dateday').agg(
    num_repochs = ('datestr','count'),
    sum_rewards=('rewards','sum')
).reset_index().merge(df_prices,how='left',left_on='dateday',right_on='dateday')
df_reward_history_days['rewards_usd']=df_reward_history_days.sum_rewards*df_reward_history_days.price

In [16]:
#Nodecount: get all current mixnodes and then get date of first pledge
dct_mixnodes = get_mixnodes('api_guru')
df_mixnodes=pd.DataFrame.from_dict(dct_mixnodes)

In [42]:
df_mixnodes['first_pledge']=np.nan
for i in range(0,len(df_mixnodes)):

    a = df_mixnodes.iloc[i].address
    dct_out=get_account_pledge(a)
    if len(dct_out)>0:
        blockdate = get_block_date(dct_out[0]['block'])
        delta = len(blockdate[0]['time'])-20
        blocktime = convert_blocktime(blockdate[0]['time'],delta=delta)
        df_mixnodes.loc[df_mixnodes.address==a,'first_pledge'] = blocktime
    

https://mixnet.api.explorers.guru/api/accounts/n12fxasranak2spt8cge0erp6293c7l4wzpapy22/bondings
https://mixnet.api.explorers.guru/api/blocks?count=1&height=3728919
https://mixnet.api.explorers.guru/api/accounts/n12j32wqhl3jncf0kz0r34xtjg07m3dfdsmqgzav/bondings
https://mixnet.api.explorers.guru/api/blocks?count=1&height=4038766
https://mixnet.api.explorers.guru/api/accounts/n12k46kt9v6lx78npgeuws2s5ggxfwwhxecze56n/bondings
https://mixnet.api.explorers.guru/api/blocks?count=1&height=4951651
https://mixnet.api.explorers.guru/api/accounts/n12psvxg5je3eg4xj47mu52tv3hpfjs2sz0alvv2/bondings
https://mixnet.api.explorers.guru/api/blocks?count=1&height=5591412
https://mixnet.api.explorers.guru/api/accounts/n12rs8e4xt2uuyc8xs30h79pdx4fl5g46uzu9w9p/bondings
https://mixnet.api.explorers.guru/api/blocks?count=1&height=1418503
https://mixnet.api.explorers.guru/api/accounts/n12twjv02qrdy2ny0cnsxjqdz4dwnu08yyqw6u42/bondings
https://mixnet.api.explorers.guru/api/blocks?count=1&height=2951838
https://mi

In [45]:
df_mixnodes['first_pledge_month'] = [str(df_mixnodes.iloc[i].first_pledge)[0:7] for i in range(0,len(df_mixnodes))]

In [47]:
df_node_count = df_mixnodes.groupby('first_pledge_month').agg({'id':'nunique'}).reset_index()
df_node_count['cs_count'] = df_node_count.id.cumsum()

In [24]:
#node level details - current data
conversion_factor = np.power(10,6) #decimals of NYM
df_nodes = read_mixnodes(conversion_factor=conversion_factor)


https://validator.nymtech.net/api/v1/mixnodes/
https://mixnet.api.explorers.guru/api/mixnodes
https://mixnet.api.explorers.guru/api/accounts/n1cg6lgel27zmq8yutxgnu2tdzx3hf0m8uemfpwp/balance
https://mixnet.api.explorers.guru/api/mixnodes/1/estimated_reward
https://mixnet.api.explorers.guru/api/mixnodes/1/uptime
https://116.203.115.86:8000/stats
HTTPSConnectionPool(host='116.203.115.86', port=8000): Max retries exceeded with url: /stats (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1131)')))
None
https://mixnet.api.explorers.guru/api/accounts/n1vfdtzh9vzaaxxv9n9w4ywdzle9m2pejhx0d5r7/balance
https://mixnet.api.explorers.guru/api/mixnodes/2/estimated_reward
https://mixnet.api.explorers.guru/api/mixnodes/2/uptime
https://95.179.226.75:8000/stats
HTTPSConnectionPool(host='95.179.226.75', port=8000): Max retries exceeded with url: /stats (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1131)')))
None
https://

In [25]:
df_nodes=df_nodes.drop(columns=['selfBonded'])
df_nodes['hours_since_last_touched']=[float((datetime.now()-df_nodes.iloc[i].last_touched).days)/24 for i in range(0,len(df_nodes))]
df_nodes['stake_till_saturation']=(df_nodes.current_bond+df_nodes.delegate_stake)*(1/(df_nodes.saturation/100)-1)
df_nodes.loc[df_nodes['stake_till_saturation']<0,'stake_till_saturation']=0
df_nodes['delegate_reward_per_k_nym']=0
df_nodes.loc[df_nodes.total_node_reward_estimate-df_nodes.cost_hour>0, 'delegate_reward_per_k_nym']=(1-df_nodes.profit_margin)*1000/(df_nodes.current_bond+df_nodes.delegate_stake)*(df_nodes.total_node_reward_estimate-df_nodes.cost_hour)
df_nodes['delegate_reward_per_k_nym_check']=df_nodes.delegates_estimate/df_nodes.delegate_stake*1000
df_nodes['rank_delegate_reward']=df_nodes.delegate_reward_per_k_nym.rank(ascending=False)

In [26]:
date_str = '2023-06-19'
df_nodes.to_pickle('nym_nodes_' + date_str + '.pkl')
df_nodes.to_csv('node_data_' + date_str + '.csv')

In [27]:
df_nodes

,id,owner,rank,status,verify,warning,warning_description,saturation,current_bond,last_touched,...,delegatedLiquid,claimable,claimableCommission,claimableRewards,claimableUnlocked,hours_since_last_touched,stake_till_saturation,delegate_reward_per_k_nym,delegate_reward_per_k_nym_check,rank_delegate_reward
0,1,n1cg6lgel27zmq8yutxgnu2tdzx3hf0m8uemfpwp,168,inactive,False,False,,0.242759,182.267768,2023-06-19 09:01:15,...,0.000000,19.099283,18.081553,1.017730,0.0,0.0,938784.962780,0.000000,0.000000,354.0
1,2,n1vfdtzh9vzaaxxv9n9w4ywdzle9m2pejhx0d5r7,135,active,False,True,This node has been impersonating Nodes.Guru an...,68.529426,2844.316263,2023-06-19 09:01:15,...,0.000000,1152.359851,1152.359851,0.000000,0.0,0.0,296159.970870,0.017811,0.017768,218.0
2,5,n1v7nrjw6pnkrdxmmuk7pjtxrzrxrtztgh33pszu,1,active,False,False,,164.088609,35750.106604,2023-06-19 09:01:15,...,0.000000,6997.703159,6997.703159,0.000000,0.0,0.0,0.000000,0.012764,0.012763,239.0
3,6,n1kal4zg6d4xe8a9uygz4vh3ngyqgpjud53m0n9w,173,active,False,False,,23.002355,5048.117226,2023-06-19 09:01:15,...,0.000000,44.117253,44.117253,0.000000,0.0,0.0,724601.347529,0.019921,0.019918,141.0
4,8,n1hn0fkhequgwdxw3d600jhax76j07kqdxklvcwn,167,active,False,False,,25.583037,2506.664010,2023-06-19 09:01:15,...,6297.209846,221.131821,141.800527,79.331293,0.0,0.0,700315.330626,0.020566,0.020561,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,987,n1m6fswhet2ascksageq0yqyt9x3d0t7hcu95929,48,inactive,False,False,,3.833045,36071.618974,2023-06-19 09:01:15,...,0.000000,21.618974,21.618974,0.000000,0.0,0.0,904997.870374,0.000000,NaN,354.0
463,990,n1zar3ankmyr0wgyrueve24xryfz285ehpmnm8q6,251,inactive,False,False,,0.027735,261.005704,2023-06-19 09:01:15,...,0.000000,0.005704,0.005704,0.000000,0.0,0.0,940808.483643,0.000000,NaN,354.0
464,991,n10amq96xgpjeppa95rj0gtffgk7jr20l5cxfhjn,252,inactive,False,False,,0.027735,261.005589,2023-06-19 09:01:15,...,0.000000,0.005589,0.005589,0.000000,0.0,0.0,940808.483759,0.000000,NaN,354.0
465,992,n1d86p63vkjdnm2k4td248xddd7z5h2nlgfua239,220,inactive,False,False,,0.106262,1000.000000,2023-06-19 09:01:15,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,940069.489348,0.000000,NaN,354.0
